In [1]:
library(tableone)
library(Matching)
library(MatchIt)

Warning message:
"package 'tableone' was built under R version 3.3.3"Warning message:
"package 'Matching' was built under R version 3.3.3"Loading required package: MASS
## 
##  Matching (Version 4.9-2, Build Date: 2015-12-25)
##  See http://sekhon.berkeley.edu/matching for additional documentation.
##  Please cite software as:
##   Jasjeet S. Sekhon. 2011. ``Multivariate and Propensity Score Matching
##   Software with Automated Balance Optimization: The Matching package for R.''
##   Journal of Statistical Software, 42(7): 1-52. 
##

Warning message:
"package 'MatchIt' was built under R version 3.3.3"

Now load the lalonde data (which is in the MatchIt package)

In [2]:
data(lalonde)

In [3]:
str(lalonde)

'data.frame':	614 obs. of  10 variables:
 $ treat   : int  1 1 1 1 1 1 1 1 1 1 ...
 $ age     : int  37 22 30 27 33 22 23 32 22 33 ...
 $ educ    : int  11 9 12 11 8 9 12 11 16 12 ...
 $ black   : int  1 0 1 1 1 1 1 1 1 0 ...
 $ hispan  : int  0 1 0 0 0 0 0 0 0 0 ...
 $ married : int  1 0 0 0 0 0 0 0 0 1 ...
 $ nodegree: int  1 1 0 1 1 1 0 1 0 0 ...
 $ re74    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ re75    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ re78    : num  9930 3596 24909 7506 290 ...


In [7]:
library(dplyr)

Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following object is masked from 'package:MASS':

    select

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



1. Find the standardized differences for all of the confounding variables (pre-matching). 
$
smd = \frac{\bar{x_t} - \bar{x_c}}{\sqrt{\frac{sd(x_t)^2 + sd(x_c)^2}{2}}}
$

In [17]:
xvars <- c('age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75')

table1 <- CreateTableOne(vars = xvars, strata = 'treat', data = lalonde, test = FALSE)

print(table1, smd = TRUE)

                      Stratified by treat
                       0                 1                 SMD   
  n                        429               185                 
  age (mean (sd))        28.03 (10.79)     25.82 (7.16)     0.242
  educ (mean (sd))       10.24 (2.86)      10.35 (2.01)     0.045
  black (mean (sd))       0.20 (0.40)       0.84 (0.36)     1.668
  hispan (mean (sd))      0.14 (0.35)       0.06 (0.24)     0.277
  married (mean (sd))     0.51 (0.50)       0.19 (0.39)     0.719
  nodegree (mean (sd))    0.60 (0.49)       0.71 (0.46)     0.235
  re74 (mean (sd))     5619.24 (6788.75) 2095.57 (4886.62)  0.596
  re75 (mean (sd))     2466.48 (3292.00) 1532.06 (3219.25)  0.287


 $2$. What is the raw (unadjusted) mean of real earnings in 1978 for treated subjects minus the mean of real earnings in 1978 for untreated subjects?

In [18]:
mean(lalonde[lalonde$treat == 1, 're78']) - mean(lalonde[lalonde$treat == 0, 're78'])

[1] -635.0262

Fit a propensity score model. Use a logistic regression model, where the outcome is treatment. Include the 8 confounding variables in the model as predictors, with no interaction terms or non-linear terms (such as squared terms). Obtain the propensity score for each subject.

In [19]:
psmodel <- glm(treat ~ age + educ + black + hispan + married + nodegree + re74 + re75, family = binomial(), data = lalonde)
pscore <- psmodel$fitted.values

$3$. What are the minimum and maximum values of the estimated propensity score?

In [20]:
summary(pscore)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
0.00908 0.04854 0.12070 0.30130 0.63870 0.85320 

Now carry out propensity score matching using the Match function.

Before using the Match function, first do:

$>set.seed(931139)$

Setting the seed will ensure that you end up with a matched data set that is the same as the one used to create the solutions.

Use options to specify pair matching, without replacement, no caliper.

Match on the propensity score itself, not logit of the propensity score. Obtain the standardized differences for the matched data.

In [21]:
set.seed(931139)

In [32]:
psmatch <- Match(Tr = lalonde$treat, M = 1, X = pscore, replace = FALSE)

matched1 <- lalonde[unlist(psmatch[c('index.treated', 'index.control')]), ]

matchedtab1 <-  CreateTableOne(vars = xvars, strata = 'treat', data = matched1, test = FALSE)

print(matchedtab1, smd = TRUE)

                      Stratified by treat
                       0                 1                 SMD   
  n                        185               185                 
  age (mean (sd))        25.58 (10.71)     25.82 (7.16)     0.026
  educ (mean (sd))       10.52 (2.69)      10.35 (2.01)     0.073
  black (mean (sd))       0.47 (0.50)       0.84 (0.36)     0.852
  hispan (mean (sd))      0.23 (0.42)       0.06 (0.24)     0.491
  married (mean (sd))     0.22 (0.42)       0.19 (0.39)     0.080
  nodegree (mean (sd))    0.65 (0.48)       0.71 (0.46)     0.116
  re74 (mean (sd))     2432.75 (4334.49) 2095.57 (4886.62)  0.073
  re75 (mean (sd))     1673.05 (2680.42) 1532.06 (3219.25)  0.048


Re-do the matching, but use a caliper this time. Set the caliper=0.1 in the options in the Match function.

In [36]:
set.seed(931139)

In [37]:
psmatch <- Match(Tr = lalonde$treat, M = 1, X = pscore, replace = FALSE, caliper = 0.1)

matched2 <- lalonde[unlist(psmatch[c('index.treated', 'index.control')]), ]

matchedtab2 <-  CreateTableOne(vars = xvars, strata = 'treat', data = matched2, test = FALSE)

print(matchedtab2, smd = TRUE)

                      Stratified by treat
                       0                 1                 SMD   
  n                        111               111                 
  age (mean (sd))        26.27 (11.10)     26.22 (7.18)     0.006
  educ (mean (sd))       10.37 (2.66)      10.25 (2.31)     0.047
  black (mean (sd))       0.72 (0.45)       0.74 (0.44)     0.040
  hispan (mean (sd))      0.11 (0.31)       0.10 (0.30)     0.029
  married (mean (sd))     0.24 (0.43)       0.24 (0.43)    <0.001
  nodegree (mean (sd))    0.66 (0.48)       0.65 (0.48)     0.019
  re74 (mean (sd))     2704.56 (4759.89) 2250.49 (5746.14)  0.086
  re75 (mean (sd))     1969.10 (3169.08) 1222.25 (3081.19)  0.239


$7$. For the matched data, what is the mean of real earnings in 1978 for treated subjects minus the mean of real earnings in 1978 for untreated subjects?

In [34]:
mean(matched1[matched1$treat == 1, 're78']) - mean(matched1[matched1$treat == 0, 're78'])

[1] 733.8864

In [38]:
mean(matched2[matched2$treat == 1, 're78']) - mean(matched2[matched2$treat == 0, 're78'])

[1] 1246.806

In [39]:
re_trt <- matched2$re78[matched2$treat == 1]
re_ctl <- matched2$re78[matched2$treat == 0]

In [40]:
re_diff = re_trt - re_ctl

In [41]:
t.test(re_diff)


	One Sample t-test

data:  re_diff
t = 1.4824, df = 110, p-value = 0.1411
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -420.0273 2913.6398
sample estimates:
mean of x 
 1246.806 


In [43]:
t.test(re78 ~ treat, paired =  TRUE, data = matched2)


	Paired t-test

data:  re78 by treat
t = -1.4824, df = 110, p-value = 0.1411
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -2913.6398   420.0273
sample estimates:
mean of the differences 
              -1246.806 


In [45]:
t.test(x = re_ctl, y = re_trt, paired = TRUE)


	Paired t-test

data:  re_ctl and re_trt
t = -1.4824, df = 110, p-value = 0.1411
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -2913.6398   420.0273
sample estimates:
mean of the differences 
              -1246.806 
